In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Saeel's Link
df = pd.read_csv("/content/drive/MyDrive/DS203/EDA.csv")

Link to the dataset: https://drive.google.com/file/d/1L6yQXX1DntaIWhKMIviJnCoM2FIF8prM/view?usp=share_link

If Colab is crashing, we can reduce the size of our dataset by the command below.

In [ ]:
df.drop(df.tail(120000).index, inplace= True)

# **Splitting our Dataset**

In [ ]:
x = df['Description']
y = df['OpenStatus']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [ ]:
vectorizer = TfidfVectorizer(max_features = 10000) 
x_train_arr = vectorizer.fit_transform(x_train).toarray()  
x_test_arr = vectorizer.transform(x_test).toarray() 

# **Model Building**

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 

# **Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(x_train_arr, y_train)

RandomForestClassifier()

In [ ]:
y_predicted_test4 = rfc.predict(x_test_arr)

In [ ]:
print("Testing Accuracy score:",accuracy_score(y_test, y_predicted_test4))

Testing Accuracy score: 0.5756330154554423


In [ ]:
print('Confusion matrix\n', confusion_matrix(y_test, y_predicted_test4))

Confusion matrix
 [[ 381   29    1  900    0]
 [  77  167    3  389    0]
 [  90   35    6  663    0]
 [ 138   16    0 2947    0]
 [  18    1    0  221    0]]


# **XGBoost Classifier**

In [ ]:
from xgboost import XGBClassifier

Reference: https://xgboost.readthedocs.io/en/stable/get_started.html

In [ ]:
xgb = XGBClassifier(objective='multi:softmax')

In [ ]:
xgb.fit(x_train_arr, y_train)

XGBClassifier(objective='multi:softprob')

In [ ]:
y_predicted_test5 = xgb.predict(x_test_arr)

In [ ]:
print("Testing Accuracy score:",accuracy_score(y_test, y_predicted_test5))

Testing Accuracy score: 0.5910884577441631


In [ ]:
print('Confusion matrix\n', confusion_matrix(y_test, y_predicted_test5))

Confusion matrix
 [[ 405   43   15  846    2]
 [  79  234   18  305    0]
 [ 100   72   74  548    0]
 [ 157   39   25 2880    0]
 [  16    0    1  221    2]]


**Proceed only if the RAM is sufficient to handle Hyperparamater tuning without crashing**

# **Hyperparameter Tuning**

In [ ]:
from sklearn.model_selection import GridSearchCV

Reference: https://xgboost.readthedocs.io/en/stable/parameter.html

In [ ]:
est = XGBClassifier(objective='multi:softmax')   #### As we are doing multi-class classification, we are using multi-softmax

In [ ]:
### We define the range for each of the tunable parameters of XGBoost

In [ ]:
params = {'min_child_weight': [1, 3, 15],
         'gamma': [0, 0.5, 1, 1.5],
         'subsample': [0.5, 0.7, 0.9],
         'colsample_bytree': [0.6, 0.8, 1.0],
         'max_depth': [4, 6, 8]
         }

In [ ]:
gsv = GridSearchCV(estimator= est, param_grid= params, scoring = 'accuracy', n_jobs = 10, cv = 2, verbose=True)

In [ ]:
gsv.fit(x_train_arr, y_train)

Fitting 2 folds for each of 324 candidates, totalling 648 fits


ERROR:concurrent.futures:exception calling callback for <Future at 0x7fda57597b50 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/_base.py", line 26, in _invoke_callbacks
    callback(self)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 385, in __call__
    self.parallel.dispatch_next()
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 834, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 901, in dispatch_one_batch
    self._dispatch(tasks)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 819, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/usr/local/lib/python3.7/dist-packages/joblib/_parallel_backends.py", line 556, in apply_async
    future = self._workers.submit(SafeFunction(func))

TerminatedWorkerError: ignored

In [ ]:
print(gsv.best_params_)     #### Prints the best combination of the parameters

In [ ]:
y_predicted_test7 = gsv.predict(x_test_arr)

In [ ]:
print("Testing Accuracy score:",accuracy_score(y_test, y_predicted_test5))

In [ ]:
print('Confusion matrix\n', confusion_matrix(y_test, y_predicted_test5))